In [10]:
# # from google.colab import auth
# # auth.authenticate_user()

# from google.colab import drive

# # This mounts your Google Drive
# drive.mount('/content/drive')
# # drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# !git clone https://github.com/jafetgado/EpHod.git

# # # Cloning into 'EpHod'...
# # # remote: Enumerating objects: 355, done.
# # # remote: Counting objects: 100% (85/85), done.
# # # remote: Compressing objects: 100% (55/55), done.
# # # remote: Total 355 (delta 47), reused 46 (delta 30), pack-reused 270 (from 1)
# # # Receiving objects: 100% (355/355), 59.95 MiB | 21.17 MiB/s, done.
# # # Resolving deltas: 100% (162/162), done.


Cloning into 'EpHod'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 355 (delta 47), reused 46 (delta 30), pack-reused 270 (from 1)
Receiving objects: 100% (355/355), 59.95 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [12]:
%cd EpHod

# # /content/EpHod

/content/EpHod/EpHod/EpHod


In [13]:
!ls

# #env.yml  ephod	example  LICENSE  README.rst

env.yml  ephod	example  LICENSE  README.rst


[`env.yml`](https://github.com/jafetgado/EpHod/blob/master/env.yml):
```
name: env
channels:
  - conda-forge
  - bioconda
  - pytorch
  - defaults
dependencies:
  - python=3.8.8
  - pytorch=1.7.0
  - wget
  - pip
  - pip:
    - fair-esm==1.0.2
    - transformers
    - sentencepiece
    - h5py
    - scikit-learn==0.23.2
    - numpy==1.19.1
    - biopython==1.79
    - pandas==1.1.3
    - xlrd
    - openpyxl
    - matplotlib
    - ipython
    - tqdm
```

```

In [14]:
!sed -n '1,200p' env.yml


# # name: env
# # channels:
# #   - conda-forge
# #   - bioconda
# #   - pytorch
# #   - defaults
# # dependencies:
# #   - python=3.8.8
# #   - pytorch=1.7.0
# #   - wget
# #   - pip
# #   - pip:
# #     - fair-esm==1.0.2
# #     - transformers
# #     - sentencepiece
# #     - h5py
# #     - scikit-learn==0.23.2
# #     - numpy==1.19.1
# #     - biopython==1.79
# #     - pandas==1.1.3
# #     - xlrd
# #     - openpyxl
# #     - matplotlib
# #     - ipython
# #     - tqdm

name: env
channels:
  - conda-forge
  - bioconda
  - pytorch
  - defaults
dependencies:
  - python=3.8.8
  - pytorch=1.7.0
  - wget
  - pip
  - pip:
    - fair-esm==1.0.2
    - transformers
    - sentencepiece
    - h5py
    - scikit-learn==0.23.2
    - numpy==1.19.1
    - biopython==1.79
    - pandas==1.1.3
    - xlrd
    - openpyxl
    - matplotlib
    - ipython
    - tqdm

[change runtime to GPU to that processing takes just ~10 minutes]

In [15]:
# --- system sanity ---
!python -V
import torch, sys
print("torch:", torch.__version__)
print("python:", sys.version)


# # Python 3.12.12
# # torch: 2.9.0+cu126
# # python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]



Python 3.12.12
torch: 2.9.0+cu126
python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [16]:

# --- clone repo ---
%cd /content
!rm -rf EpHod
!git clone https://github.com/jafetgado/EpHod.git
%cd /content/EpHod



# # /content
# # Cloning into 'EpHod'...
# # remote: Enumerating objects: 355, done.
# # remote: Counting objects: 100% (85/85), done.
# # remote: Compressing objects: 100% (55/55), done.
# # remote: Total 355 (delta 47), reused 46 (delta 30), pack-reused 270 (from 1)
# # Receiving objects: 100% (355/355), 59.95 MiB | 15.10 MiB/s, done.
# # Resolving deltas: 100% (162/162), done.
# # /content/EpHod


/content
Cloning into 'EpHod'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 355 (delta 47), reused 46 (delta 30), pack-reused 270 (from 1)
Receiving objects: 100% (355/355), 59.95 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/EpHod


In [17]:

# --- pip deps (pip-first) ---
# fair-esm is the ESM package listed in env.yml (pinned there to 1.0.2).
# We'll start with that and modern wheels for the rest.
!pip -q install --upgrade pip

!pip -q install \
  fair-esm==1.0.2 \
  transformers sentencepiece h5py \
  biopython pandas openpyxl xlrd \
  tqdm matplotlib ipython



#   #  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.2 MB/s eta 0:00:00



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.2 MB/s eta 0:00:00


In [18]:
# Make EpHod importable
import os, sys
sys.path.append("/content/EpHod")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [20]:

FASTA_PATH = "/content/drive/MyDrive/SynBio Software/data/processed/predictive_pet_zero_shot_2025.fasta"
!ls -lh "{FASTA_PATH}"
!head -n 6 "{FASTA_PATH}"



# # -rw------- 1 root root 1.3M Jan  7 19:23 '/content/drive/MyDrive/SynBio Software/data/processed/predictive_pet_zero_shot_2025.fasta'
# # >seq_0
# # AADNPYQRGPDPTNASIEAATGPFAAGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF
# # >seq_1
# # AADNPYQRGPDPTNASIEAATGPFACGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF
# # >seq_2
# # AADNPYQRGPDPTNASIEAATGPFADGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF


-rw------- 1 root root 1.3M Jan  7 19:23 '/content/drive/MyDrive/SynBio Software/data/processed/predictive_pet_zero_shot_2025.fasta'
>seq_0
AADNPYQRGPDPTNASIEAATGPFAAGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF
>seq_1
AADNPYQRGPDPTNASIEAATGPFACGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF
>seq_2
AADNPYQRGPDPTNASIEAATGPFADGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF


In [21]:
# Run EpHod (writes a CSV)
SAVE_DIR = "/content/drive/MyDrive/SynBio Software/data/processed/ephod_outputs_4988"
!mkdir -p "{SAVE_DIR}"

!python ./ephod/run.py \
  --fasta_path "{FASTA_PATH}" \
  --save_dir "{SAVE_DIR}" \
  --csv_name "ephod_prediction_4988.csv" \
  --verbose 1 \
  --save_attention_weights 0 \
  --save_embeddings 0


# # Reading 4988 sequences from /content/drive/MyDrive/SynBio Software/data/processed/predictive_pet_zero_shot_2025.fasta
# # Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1v_t33_650M_UR90S_1.pt" to /root/.cache/torch/hub/checkpoints/esm1v_t33_650M_UR90S_1.pt
# # Downloading: "https://zenodo.org/records/14252615/files/ESM1v-RLATtr.pt?download=1" to /root/.cache/torch/hub/checkpoints/ESM1v-RLATtr.pt
# # Initializing EpHod model
# # Device is cuda
# # Predicting pHopt: 100% 4988/4988 [15:44<00:00,  5.28it/s]
# # Prediction completed.


Reading 4988 sequences from /content/drive/MyDrive/SynBio Software/data/processed/predictive_pet_zero_shot_2025.fasta
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1v_t33_650M_UR90S_1.pt" to /root/.cache/torch/hub/checkpoints/esm1v_t33_650M_UR90S_1.pt
Downloading: "https://zenodo.org/records/14252615/files/ESM1v-RLATtr.pt?download=1" to /root/.cache/torch/hub/checkpoints/ESM1v-RLATtr.pt
Initializing EpHod model
Device is cuda
Predicting pHopt: 100% 4988/4988 [15:44<00:00,  5.28it/s]
Prediction completed.


In [22]:
# When the run completes, run:

!ls -lh "{SAVE_DIR}"
!head -n 5 "{SAVE_DIR}/ephod_prediction_4988.csv"


# # total 305K
# # -rw------- 1 root root 305K Jan 21 01:31 ephod_prediction_4988.csv
# # ,RLATtr,SVR,Ensemble
# # seq_0,7.73985481262207,7.715606184470648,7.727730498546359
# # seq_1,7.725358963012695,7.669894179080865,7.69762657104678
# # seq_2,7.723910331726074,7.71303476887101,7.718472550298542
# # seq_3,7.7326812744140625,7.730927671967294,7.731804473190678



total 305K
-rw------- 1 root root 305K Jan 21 01:31 ephod_prediction_4988.csv
,RLATtr,SVR,Ensemble
seq_0,7.73985481262207,7.715606184470648,7.727730498546359
seq_1,7.725358963012695,7.669894179080865,7.69762657104678
seq_2,7.723910331726074,7.71303476887101,7.718472550298542
seq_3,7.7326812744140625,7.730927671967294,7.731804473190678


now let's inspect the scores

In [26]:
import pandas as pd

pred_path = f"{SAVE_DIR}/ephod_prediction_4988.csv"
pred = pd.read_csv(pred_path)

print("rows:", len(pred))
print("columns:", list(pred.columns))
print(pred.head(20))



# # rows: 4988
# # columns: ['Unnamed: 0', 'RLATtr', 'SVR', 'Ensemble']
# #    Unnamed: 0    RLATtr       SVR  Ensemble
# # 0       seq_0  7.739855  7.715606  7.727730
# # 1       seq_1  7.725359  7.669894  7.697627
# # 2       seq_2  7.723910  7.713035  7.718473
# # 3       seq_3  7.732681  7.730928  7.731804
# # 4       seq_4  7.714048  7.701282  7.707665
# # 5       seq_5  7.745150  7.704092  7.724621
# # 6       seq_6  7.754671  7.680321  7.717496
# # 7       seq_7  7.734118  7.717221  7.725670
# # 8       seq_8  7.723994  7.671086  7.697540
# # 9       seq_9  7.724413  7.711132  7.717772
# # 10     seq_10  7.731717  7.701877  7.716797
# # 11     seq_11  7.698694  7.671348  7.685021
# # 12     seq_12  7.751860  7.681650  7.716755
# # 13     seq_13  7.747885  7.712959  7.730422
# # 14     seq_14  7.753529  7.750568  7.752049
# # 15     seq_15  7.733411  7.702696  7.718054
# # 16     seq_16  7.740658  7.712987  7.726822
# # 17     seq_17  7.707484  7.717899  7.712691
# # 18     seq_18  7.673983  7.644418  7.659200
# # 19     seq_19  7.726765  7.723907  7.725336


rows: 4988
columns: ['Unnamed: 0', 'RLATtr', 'SVR', 'Ensemble']
   Unnamed: 0    RLATtr       SVR  Ensemble
0       seq_0  7.739855  7.715606  7.727730
1       seq_1  7.725359  7.669894  7.697627
2       seq_2  7.723910  7.713035  7.718473
3       seq_3  7.732681  7.730928  7.731804
4       seq_4  7.714048  7.701282  7.707665
5       seq_5  7.745150  7.704092  7.724621
6       seq_6  7.754671  7.680321  7.717496
7       seq_7  7.734118  7.717221  7.725670
8       seq_8  7.723994  7.671086  7.697540
9       seq_9  7.724413  7.711132  7.717772
10     seq_10  7.731717  7.701877  7.716797
11     seq_11  7.698694  7.671348  7.685021
12     seq_12  7.751860  7.681650  7.716755
13     seq_13  7.747885  7.712959  7.730422
14     seq_14  7.753529  7.750568  7.752049
15     seq_15  7.733411  7.702696  7.718054
16     seq_16  7.740658  7.712987  7.726822
17     seq_17  7.707484  7.717899  7.712691
18     seq_18  7.673983  7.644418  7.659200
19     seq_19  7.726765  7.723907  7.725336


In [27]:
from Bio import SeqIO

fasta_ids = [r.id for r in SeqIO.parse(FASTA_PATH, "fasta")]
print("FASTA count:", len(fasta_ids), "unique:", len(set(fasta_ids)))

# Guess an ID column: often first column, but we'll be safe:
# pick the column that looks like ids if present
possible_id_cols = [c for c in pred.columns if "id" in c.lower() or "sid" in c.lower() or "seq" in c.lower()]
print("possible id cols:", possible_id_cols)

id_col = possible_id_cols[0] if possible_id_cols else pred.columns[0]

print("pred unique ids:", pred[id_col].astype(str).nunique())

missing = set(fasta_ids) - set(pred[id_col].astype(str))
extra   = set(pred[id_col].astype(str)) - set(fasta_ids)

print("missing from predictions:", len(missing))
print("extra in predictions:", len(extra))





# # FASTA count: 4988 unique: 4988
# # possible id cols: []
# # pred unique ids: 4988
# # missing from predictions: 0
# # extra in predictions: 0



FASTA count: 4988 unique: 4988
possible id cols: []
pred unique ids: 4988
missing from predictions: 0
extra in predictions: 0
